In [1]:
import cv2
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as Poly

In [2]:
dir_location = '/scratch/ichakr2s/bdd100k/data/labels/10k/ins_seg/polygons/'
train_location = '/scratch/ichakr2s/bdd100k/data/images/10k/train/'
output_location = '/scratch/ichakr2s/yolov5pytorch_aug/is_bdd10k/aug_images_by_category/'

In [3]:
with open(dir_location + 'ins_seg_train.json', 'r') as f:
    bdd_data = json.load(f)

In [9]:
bdd10k_category_list = []
for img in bdd_data:
  for properties in img['labels']:
    if properties['category'] not in bdd10k_category_list:
      bdd10k_category_list.append(properties['category'])

In [5]:
file_names = []
for idx in range(len(bdd_data)):
    for lab in bdd_data[idx]['labels']:
        if lab['category'] == 'person':
            file_names.append(bdd_data[idx])
            break

In [15]:
trname = file_names[0]['name']
img = cv2.imread(train_location + trname)
img_height = img.shape[0]
img_width = img.shape[1]

In [18]:
def make_label(file_name_str, train_img):
    strinG = ''
    for label in train_img['labels']:
        poly = Polygon(label['poly2d'][0]['vertices'])
        cat = bdd10k_category_list.index(label['category'])
        centr_xy =  poly.centroid.xy
        centr_x = centr_xy[0][0]/img_width
        centr_y = centr_xy[1][0]/img_height
        minx, miny, maxx, maxy = poly.bounds
        width, height = (maxx - minx)/img_width, (maxy - miny)/img_height
        if height > 1.0:
            height = 0.999999
        if width > 1.0:
            width = 0.999999
        if centr_x > 1.0:
            centr_x = 0.999999
        if centr_y > 1.0:
            centr_y = 0.999999
        strinG += str(cat) + ' ' + str(centr_x) + ' ' + str(centr_y) + ' ' + str(width) + ' ' + str(height)  + '\n'

    with open(output_location + 'synth_labels/' + file_name_str + '.txt' , 'w') as f:
        f.write(strinG)

def swap(i1, i2, idx_1_2, swap_idx):
    
    image1 = cv2.imread(train_location + i1['name'])
    annot1 = i1['labels'][idx_1_2[swap_idx][0]]
    roi_corners1 = np.array([annot1['poly2d'][0]['vertices']], dtype=np.int32)
    roi_mean1 = np.mean(roi_corners1[0], axis=0)
    roi_norm1 = roi_corners1[0] - roi_mean1
    xmin1 = np.min(roi_corners1[0][:,0])
    xmax1 = np.max(roi_corners1[0][:,0])
    ymin1 = np.min(roi_corners1[0][:,1])
    ymax1 = np.max(roi_corners1[0][:,1])
    
    image2 = cv2.imread(train_location + i2['name'])
    annot2 = i2['labels'][idx_1_2[swap_idx][1]]
    roi_corners2 = np.array([annot2['poly2d'][0]['vertices']], dtype=np.int32)
    roi_mean2 = np.mean(roi_corners2[0], axis=0)
    roi_norm2 = roi_corners2[0] - roi_mean2
    xmin2 = np.min(roi_corners2[0][:,0])
    xmax2 = np.max(roi_corners2[0][:,0])
    ymin2 = np.min(roi_corners2[0][:,1])
    ymax2 = np.max(roi_corners2[0][:,1])
    
    mask1 = np.ones(image1.shape, dtype=np.uint8)
    mask1.fill(255)
    cv2.fillPoly(mask1, roi_corners1, 0)
    masked_image1 = cv2.bitwise_or(image1, mask1)
    masking_obj1 = masked_image1[ymin1:ymax1, xmin1:xmax1]
    masked_image1 = np.ones(image2.shape, dtype=np.uint8)
    masked_image1.fill(255)
    masking_obj1 = cv2.resize(masking_obj1, (xmax2-xmin2, ymax2-ymin2), interpolation = cv2.INTER_AREA)
    masked_image1[ymin2:ymax2, xmin2:xmax2] = masking_obj1
    img2_mask = np.vstack((((xmax2-xmin2)/float(xmax1-xmin1))*roi_norm1[:,0], ((ymax2-ymin2)/float(ymax1-ymin1))*roi_norm1[:,1])).T    
    mask2 = np.ones(image2.shape, dtype=np.uint8)
    mask2.fill(255)
    cv2.fillPoly(mask2, np.array([img2_mask+roi_mean2], dtype=np.int), 0)
    mask2 = 255 - mask2
    mask2 = cv2.blur(mask2,(5,5))
    masked_image2 = cv2.bitwise_or(image2, mask2)
    swapped_1 = cv2.bitwise_and(masked_image1, masked_image2)
    
    mask2 = np.ones(image2.shape, dtype=np.uint8)
    mask2.fill(255)
    cv2.fillPoly(mask2, roi_corners2, 0)
    masked_image2 = cv2.bitwise_or(image2, mask2)
    masking_obj2 = masked_image2[ymin2:ymax2, xmin2:xmax2]
    masked_image2 = np.ones(image1.shape, dtype=np.uint8)
    masked_image2.fill(255)
    masking_obj2 = cv2.resize(masking_obj2, (xmax1-xmin1, ymax1-ymin1), interpolation = cv2.INTER_AREA)
    masked_image2[ymin1:ymax1, xmin1:xmax1] = masking_obj2
    img1_mask = np.vstack((((xmax1-xmin1)/float(xmax2-xmin2))*roi_norm2[:,0], ((ymax1-ymin1)/float(ymax2-ymin2))*roi_norm2[:,1])).T
    mask1 = np.ones(image1.shape, dtype=np.uint8)
    mask1.fill(255)
    cv2.fillPoly(mask1, np.array([img1_mask+roi_mean1], dtype=np.int), 0)
    mask1 = 255 - mask1
    mask1 = cv2.blur(mask1,(5,5))
    masked_image1 = cv2.bitwise_or(image1, mask1)
    swapped_2 = cv2.bitwise_and(masked_image2, masked_image1)
    
    return swapped_1, swapped_2

In [21]:
f_high = 1.55
f_low = 0.45
scale_factor = 0.001

for img1 in file_names:
    for img2 in file_names:
        if img1['name'] != img2['name']:
            im1 = cv2.imread(train_location + img1['name'])
            im2 = cv2.imread(train_location + img2['name'])
            area_thresh = im1.shape[0] *  im1.shape[1] * scale_factor
            idx_1_2 = []
            for idx1, annot1 in enumerate(img1['labels']):
                if annot1['category'] == 'person':
                    shape1 = np.array(annot1['poly2d'][0]['vertices'])
                    seg1 = Polygon(shape1)
                    ann1_ratio = (np.max(shape1[:,1])-np.min(shape1[:,1]))/(np.max(shape1[:,0])-np.min(shape1[:,0]))
                    area1 = seg1.area
                    if area1 > area_thresh:
                        for idx2, annot2 in enumerate(img2['labels']):
                            if annot2['category'] == 'person':
                                shape2 = np.array(annot2['poly2d'][0]['vertices'])
                                seg2 = Polygon(shape2)
                                ann2_ratio = (np.max(shape2[:,1])-np.min(shape2[:,1]))/(np.max(shape2[:,0])-np.min(shape2[:,0]))
                                area2 = seg2.area
                                if area2 > area_thresh:
                                    f = area1/area2
                                    if abs(ann2_ratio-ann1_ratio) < 1.:
                                        if f < f_high and f > f_low:
                                            idx_1_2.append([idx1, idx2])

            idx_1_2 = np.array(idx_1_2)
            
            if len(idx_1_2) > 0:
                for swap_idx in range(len(idx_1_2)):
                    ii2, ii1 = swap(img1, img2, idx_1_2, swap_idx)
                    img_name2 = img2['name'][:-4] + '_' + str(idx_1_2[swap_idx][1]) + '__' + img1['name'][:-4] + '_' + str(idx_1_2[swap_idx][0])
                    cv2.imwrite(output_location + 'synth_images/' + img_name2 + '.jpg', ii2)
                    make_label(img_name2, img2)
                    img_name1 = img1['name'][:-4] + '_' + str(idx_1_2[swap_idx][0]) + '__' + img2['name'][:-4] + '_' + str(idx_1_2[swap_idx][1])
                    cv2.imwrite(output_location + 'synth_images/' + img_name1 + '.jpg', ii1)                  
                    make_label(img_name1, img1)


/tmp/ipykernel_280940/974710775.py:59: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv2.fillPoly(mask2, np.array([img2_mask+roi_mean2], dtype=np.int), 0)
/tmp/ipykernel_280940/974710775.py:77: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional i

ValueError: could not broadcast input array from shape (95,40,3) into shape (95,39,3)